In [13]:
!pip install jsonlines pandas datasets
!pip install zstandard



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 14.6 MB/s eta 0:00:00


In [14]:
import jsonlines
import itertools
import pandas as pd
from pprint import pprint

import datasets
from datasets import load_dataset

In [15]:
pretrained_dataset = load_dataset("EleutherAI/pile", split="train", streaming=True)

# pretrained_dataset = load_dataset("c4", "en", split="train", streaming=True)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for EleutherAI/pile contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/EleutherAI/pile
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [25]:
import json
python_dataset = [
  {
    "question": "What is Python?",
    "answer": "Python is a high-level, interpreted programming language known for its simplicity and readability."
  },
  {
    "question": "What are the key features of Python?",
    "answer": "Key features of Python include its dynamic typing, automatic memory management, and extensive standard library."
  },
  {
    "question": "How do you comment in Python?",
    "answer": "In Python, you can use the '#' symbol to create single-line comments, and ''' ... ''' or \"\"\" ... \"\"\" for multi-line comments."
  },
  {
    "question": "What is PEP 8?",
    "answer": "PEP 8 is the style guide for Python code, which recommends coding conventions to improve code readability."
  },
  {
    "question": "What are Python's data types?",
    "answer": "Python supports various data types including integers, floats, strings, lists, tuples, dictionaries, and sets."
  },
  {
    "question": "What is the difference between '==' and 'is' in Python?",
    "answer": "The '==' operator compares the values of two objects, while the 'is' operator checks if two variables refer to the same object in memory."
  },
  {
    "question": "How do you declare a variable in Python?",
    "answer": "Variables in Python are declared simply by assigning a value to a name. For example: 'x = 5'."
  },
  {
    "question": "What is a Python function?",
    "answer": "A function in Python is a block of reusable code that performs a specific task. It can take inputs, called parameters, and return outputs."
  },
  {
    "question": "How do you import a module in Python?",
    "answer": "You can import a module in Python using the 'import' keyword followed by the module name. For example: 'import math'."
  },
  {
    "question": "What is a list comprehension in Python?",
    "answer": "A list comprehension is a concise way of creating lists in Python by applying an expression to each item in an iterable."
  }
]


filename = "python_dataset.jsonl"

# Write the dataset to the JSONL file
with open(filename, "w") as f:
    for item in python_dataset:
        json.dump({"text": item}, f)  # Write each item as a JSON object
        f.write("\n")





In [26]:
n = 5
print("Python dataset:")
top_n = python_dataset[:n]  # Slice the dataset to get the top n elements
for i in top_n:
    print(i)

Python dataset:
{'question': 'What is Python?', 'answer': 'Python is a high-level, interpreted programming language known for its simplicity and readability.'}
{'question': 'What are the key features of Python?', 'answer': 'Key features of Python include its dynamic typing, automatic memory management, and extensive standard library.'}
{'question': 'How do you comment in Python?', 'answer': 'In Python, you can use the \'#\' symbol to create single-line comments, and \'\'\' ... \'\'\' or """ ... """ for multi-line comments.'}
{'question': 'What is PEP 8?', 'answer': 'PEP 8 is the style guide for Python code, which recommends coding conventions to improve code readability.'}
{'question': "What are Python's data types?", 'answer': 'Python supports various data types including integers, floats, strings, lists, tuples, dictionaries, and sets.'}


In [31]:
filename = "lamini_docs.jsonl"
instruction_dataset_df = pd.read_json(filename, lines=True)
instruction_dataset_df

,question,answer
0,What are the different types of documents avai...,"Lamini has documentation on Getting Started, A..."
1,What is the recommended way to set up and conf...,Lamini can be downloaded as a python package a...
2,How can I find the specific documentation I ne...,"You can ask this model about documentation, wh..."
3,Does the documentation include explanations of...,Our documentation provides both real-world and...
4,Does the documentation provide information abo...,External dependencies and libraries are all av...
...,...,...
1395,What is Lamini and what is its collaboration w...,Lamini is a library that simplifies the proces...
1396,How does Lamini simplify the process of access...,Lamini simplifies data access in Databricks by...
1397,What are some of the key features provided by ...,Lamini automatically manages the infrastructur...
1398,How does Lamini ensure data privacy during the...,"During the training process, Lamini ensures da..."


In [32]:
examples = instruction_dataset_df.to_dict()
text = examples["question"][0] + examples["answer"][0]
text

"What are the different types of documents available in the repository (e.g., installation guide, API documentation, developer's guide)?Lamini has documentation on Getting Started, Authentication, Question Answer Model, Python Library, Batching, Error Handling, Advanced topics, and class documentation on LLM Engine available at https://lamini-ai.github.io/."

In [33]:
if "question" in examples and "answer" in examples:
  text = examples["question"][0] + examples["answer"][0]
elif "instruction" in examples and "response" in examples:
  text = examples["instruction"][0] + examples["response"][0]
elif "input" in examples and "output" in examples:
  text = examples["input"][0] + examples["output"][0]
else:
  text = examples["text"][0]

In [34]:
prompt_template_qa = """### Question:
{question}

### Answer:
{answer}"""

In [37]:
question = examples["question"][0]
answer = examples["answer"][0]

text_with_prompt_template = prompt_template_qa.format(question=question, answer=answer)
text_with_prompt_template

"### Question:\nWhat are the different types of documents available in the repository (e.g., installation guide, API documentation, developer's guide)?\n\n### Answer:\nLamini has documentation on Getting Started, Authentication, Question Answer Model, Python Library, Batching, Error Handling, Advanced topics, and class documentation on LLM Engine available at https://lamini-ai.github.io/."

In [38]:
prompt_template_q = """### Question:
{question}

### Answer:"""

In [39]:
num_examples = len(examples["question"])
finetuning_dataset_text_only = []
finetuning_dataset_question_answer = []
for i in range(num_examples):
  question = examples["question"][i]
  answer = examples["answer"][i]

  text_with_prompt_template_qa = prompt_template_qa.format(question=question, answer=answer)
  finetuning_dataset_text_only.append({"text": text_with_prompt_template_qa})
  text_with_prompt_template_q = prompt_template_q.format(question=question)
  finetuning_dataset_question_answer.append({"question": text_with_prompt_template_q, "answer": answer})

In [40]:
pprint(finetuning_dataset_text_only[0])

{'text': '### Question:\n'
         'What are the different types of documents available in the '
         "repository (e.g., installation guide, API documentation, developer's "
         'guide)?\n'
         '\n'
         '### Answer:\n'
         'Lamini has documentation on Getting Started, Authentication, '
         'Question Answer Model, Python Library, Batching, Error Handling, '
         'Advanced topics, and class documentation on LLM Engine available at '
         'https://lamini-ai.github.io/.'}


In [41]:
pprint(finetuning_dataset_question_answer[0])

{'answer': 'Lamini has documentation on Getting Started, Authentication, '
           'Question Answer Model, Python Library, Batching, Error Handling, '
           'Advanced topics, and class documentation on LLM Engine available '
           'at https://lamini-ai.github.io/.',
 'question': '### Question:\n'
             'What are the different types of documents available in the '
             'repository (e.g., installation guide, API documentation, '
             "developer's guide)?\n"
             '\n'
             '### Answer:'}


In [42]:
with jsonlines.open(f'lamini_docs_processed.jsonl', 'w') as writer:
    writer.write_all(finetuning_dataset_question_answer)

In [43]:
finetuning_dataset_name = "lamini/lamini_docs"
finetuning_dataset = load_dataset(finetuning_dataset_name)
print(finetuning_dataset)

Generating train split:   0%|          | 0/1260 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/140 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})


# New section